In [8]:
import random
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [12]:
with open('intents.json', 'r') as file:
    intents = json.load(file)


In [13]:
patterns = []
responses = []
tags = []
for intent in intents['intents']:
    for pattern in intent['padroes']:
        patterns.append(pattern)
        responses.append(intent['respostas'])
        tags.append(intent['tag'])


In [14]:

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(patterns)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(patterns)
sequences_padded = pad_sequences(sequences, padding='post')


In [15]:
responses_onehot = np.zeros((len(tags), len(tags)), dtype='int32')
for i, tag in enumerate(tags):
    responses_onehot[i][i] = 1


In [16]:
model = keras.Sequential([
    keras.layers.Embedding(len(word_index) + 1, 128),
    keras.layers.LSTM(128),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(len(tags), activation='softmax')
])


In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [18]:
model.fit(sequences_padded, responses_onehot, epochs=100, batch_size=8)


Epoch 1/100
6/6 [==============================] - 3s 6ms/step - loss: 3.8385 - accuracy: 0.0000e+00
Epoch 2/100
6/6 [==============================] - 0s 6ms/step - loss: 3.8280 - accuracy: 0.0652
Epoch 3/100
6/6 [==============================] - 0s 6ms/step - loss: 3.8229 - accuracy: 0.0870
Epoch 4/100
6/6 [==============================] - 0s 6ms/step - loss: 3.8175 - accuracy: 0.0870
Epoch 5/100
6/6 [==============================] - 0s 6ms/step - loss: 3.8114 - accuracy: 0.1087
Epoch 6/100
6/6 [==============================] - 0s 6ms/step - loss: 3.8021 - accuracy: 0.1522
Epoch 7/100
6/6 [==============================] - 0s 6ms/step - loss: 3.7791 - accuracy: 0.1522
Epoch 8/100
6/6 [==============================] - 0s 6ms/step - loss: 3.7418 - accuracy: 0.1304
Epoch 9/100
6/6 [==============================] - 0s 6ms/step - loss: 3.6503 - accuracy: 0.1522
Epoch 10/100
6/6 [==============================] - 0s 7ms/step - loss: 3.4900 - accuracy: 0.0870
Epoch 11/100
6/6 [=======

In [19]:
def chatbot_response(user_input):
    sequence = tokenizer.texts_to_sequences([user_input])
    sequence_padded = pad_sequences(sequence, padding='post', maxlen=len(sequences_padded[0]))
    predicted_index = np.argmax(model.predict(sequence_padded), axis=-1)[0]
    tag = tags[predicted_index]
    responses_list = responses[predicted_index]
    return random.choice(responses_list)


In [20]:
while True:
    user_input = input("Você: ")
    if user_input.lower() == 'sair':
        break
    response = chatbot_response(user_input)
    print("Chatbot: " + response)


1/1 [==============================] - 0s 385ms/step
Chatbot: OlÃ¡, como posso ajudar?
1/1 [==============================] - 0s 19ms/step
Chatbot: OlÃ¡, obrigado por perguntar
1/1 [==============================] - 0s 19ms/step
Chatbot: Navegando para o mÃ³dulo de reaÃ§Ãµes adversas a medicamentos


In [6]:
import unidecode

def chatbot_response(user_input):
    user_input = unidecode.unidecode(user_input)  # Remover acentos e caracteres especiais
    sequence = tokenizer.texts_to_sequences([user_input])
    sequence_padded = pad_sequences(sequence, padding='post', maxlen=len(sequences_padded[0]))
    predicted_index = np.argmax(model.predict(sequence_padded), axis=-1)[0]
    tag = tags[predicted_index]
    responses_list = responses[predicted_index]
    return random.choice(responses_list)


ModuleNotFoundError: No module named 'unidecode'